In [1]:
import os
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

import solution_search
from ac3 import AC3
from ac4 import AC4
from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution
from depth_first_search_solver import DepthFirstSearchSolver
from experiment_details import ExperimentDetails
from parse_input import parse_xml, parse_itc2007_curriculum_based, parse_itc2007_post_enrolment
from solution_search import SolutionSearch
from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution




In [2]:
itc2007_track2_path = '.\\Datasets\\post'
itc2007_track3_path = '.\\Datasets\\curriculum'
itc2019_path = '.\\Datasets\\2019'

In [3]:
def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]    

In [4]:
itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)

# Get statistics

## Statistics for ITC2019

In [5]:
# problems_and_stats = [parse_xml(file) for file in itc2019_files]
# stats = [problem[1] for problem in problems_and_stats]
# 
# df = pd.DataFrame(stats)
# 
# df.index += 1
# df.reset_index(inplace=True,names='index')
# 
# os.makedirs('output/stats', exist_ok=True)
# df.to_csv('output/stats/itc2019_stats.csv',index=False)


## Statistics for ITC2007 Track 2

In [6]:
# problems_and_stats = [parse_itc2007_post_enrolment(file) for file in itc2007_track2_files]
# stats = [problem[1] for problem in problems_and_stats]
# 
# df = pd.DataFrame(stats)
# 
# 
# #make index 1 based and name them as their file names ex. 1,2,...
# df.index += 1
# df.reset_index(inplace=True,names='name')
# 
# df.index += 1
# df.reset_index(inplace=True,names='index')
# 
# os.makedirs('output/stats', exist_ok=True)
# df.to_csv('output/stats/itc2007_track2_stats.csv',index=False)

## Statistics for ITC2007 Track 3

In [7]:
# problems_and_stats = [parse_itc2007_curriculum_based(file) for file in itc2007_track3_files]
# stats = [problem[1] for problem in problems_and_stats]
# 
# df = pd.DataFrame(stats)
# 
# 
# df.index += 1
# df.reset_index(inplace=True,names='index')
# 
# os.makedirs('output/stats', exist_ok=True)
# df.to_csv('output/stats/itc2007_track3_stats.csv',index=False)


# Construction using depth first search

In [8]:
def plot_row_over_operation(file,operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    y = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Operation')
    ax.set_ylabel('Classes placed')
    
    ax.plot(y)
    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    
    
    
def plot_row_over_time(file, operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    time = [i['time'] for i in operation_history]
    row = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Classes placed')
    ax.plot(time,row)

    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    

In [9]:
def parse_problem (dataset,instance_path):
    if dataset == 'itc2007_track2':
        return parse_itc2007_post_enrolment(instance_path)
    elif dataset == 'itc2007_track3':
        return parse_itc2007_curriculum_based(instance_path)
    elif dataset == 'itc2019':
        return parse_xml(instance_path)


def run_experiment(experiment_name, problem_files, dataset, ac3_params = None,ac4_params=None, depth_first_params=None):
    
    folder = f'output/{experiment_name}_{dataset}_{time.strftime("%Y%m%d-%H%M%S")}'
    os.mkdir(folder)
    
    print(f'experiment results will be saved in {folder}')
    
    for f in problem_files:
        date = time.time()
        problem = parse_problem(dataset,f)[0]        
        sub_folder = os.path.join(folder, f.split('\\')[-1].split('.')[0])
        os.mkdir(sub_folder)
        
        res = {}
        
        search = None
        if ac3_params is not None or ac4_params is not None or depth_first_params is not None:
            solution_search_init_start_time = time.time()
            search = SolutionSearch(problem)
            solution_search_init_end_time = time.time()
            res['solution_search_init_time'] = solution_search_init_end_time - solution_search_init_start_time            
            
        if ac3_params is not None:
            
            ac3_res = {}
            
            ac3_start_time = time.time()
            ac3 = AC3(search)            
            ac3_res['ac3_init_time'] = time.time() - ac3_start_time
            
            ac3_apply_start_time = time.time()
            ac3_return_value = ac3.apply(debugLevel=1,**ac3_params)
            ac3_end_time = time.time()
            ac3_res['ac3_apply_time'] = ac3_end_time - ac3_apply_start_time
            
            ac3_res['ac3_total_time'] = ac3_end_time - ac3_start_time
            
            options_open_before_ac3 = np.count_nonzero(search.decision_table == 0)
            options_open_after_ac3 = np.count_nonzero(search.decision_table == 0)
            options_closed_by_ac3 = options_open_before_ac3 - options_open_after_ac3
            
            ac3_res['options_open_before_ac3'] = options_open_before_ac3
            ac3_res['options_open_after_ac3'] = options_open_after_ac3
            ac3_res['options_closed_by_ac3'] = options_closed_by_ac3
            ac3_res['ac3_return_value'] = ac3_return_value
            
            if ac3_return_value:
                pickle.dump(search.decision_table, open(os.path.join(sub_folder, 'decision_table_after_ac3.pkl'), 'wb'))
                
            res['ac3_results'] = ac3_res
        
        if ac4_params is not None:
            
            ac4_res = {}
            
            ac4_start_time = time.time()
            ac4 = AC4(search)            
            ac4_res['ac4_init_time'] = time.time() - ac4_start_time
            
            ac4_apply_start_time = time.time()
            ac4_return_value = ac4.apply(debugLevel=1,**ac4_params)
            ac4_end_time = time.time()
            ac4_res['ac4_apply_time'] = ac4_end_time - ac4_apply_start_time
            
            ac4_res['ac4_total_time'] = ac4_end_time - ac4_start_time
            
            options_open_before_ac4 = np.count_nonzero(search.decision_table == 0)
            options_open_after_ac4 = np.count_nonzero(search.decision_table == 0)
            options_closed_by_ac4 = options_open_before_ac4 - options_open_after_ac4
            
            ac4_res['options_open_before_ac4'] = options_open_before_ac4
            ac4_res['options_open_after_ac4'] = options_open_after_ac4
            ac4_res['options_closed_by_ac4'] = options_closed_by_ac4
            ac4_res['ac4_return_value'] = ac4_return_value
            
            if ac4_return_value:
                pickle.dump(search.decision_table, open(os.path.join(sub_folder, 'decision_table_after_ac4.pkl'), 'wb'))
                
            res['ac4_results'] = ac4_res
            
        if depth_first_params is not None:
            solver = DepthFirstSearchSolver(search)            
            depth_first_res = solver.solve(debug_level=1,**depth_first_params)
            
            if depth_first_res['success']:
                np.save(os.path.join(sub_folder, 'solution gene after depth first search.npy'), search.get_result_as_gene())
            
            
            instance_file_name = f.split('\\')[-1].split('.')[0]
            instance_name = problem.name if dataset == 'itc2019' else instance_file_name
            
            plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), depth_first_res['operation_history'], f"{experiment_name}\n{dataset} - {instance_name}")
            plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),depth_first_res['operation_history'], f"{experiment_name}\n{dataset} - {instance_name}")
            
            res['depth_first_search_results'] = depth_first_res
        
        
        experiment_details = ExperimentDetails(
            name=experiment_name,
            instance_name=instance_name,
            instance_file_name = instance_file_name,
            dataset_name=dataset,
            date=date,
            results_data=res,
            experiment_data={
                'ac3_params': ac3_params,
                'ac4_params': ac4_params,
                'depth_first_params': depth_first_params
            }
        )
        
        pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
        

## Depth first search with 10k operations

## AC4 followed by Depth first search with 10k operations

In [10]:
# run_experiment('Depth_first_search with 10k max operations', itc2007_track2_files, 'itc2007_track2', ac4_params=None, depth_first_params={'max_operations': 10000})

In [11]:
# run_experiment('Depth_first_search with 10k max operations', itc2007_track3_files, 'itc2007_track3', ac4_params=None, depth_first_params={'max_operations': 10000})

In [12]:
# run_experiment('Depth_first_search with 10k max operations', itc2019_files, 'itc2019', ac4_params=None, depth_first_params={'max_operations': 10000})

In [13]:
run_experiment('AC3 and depth_first_search with 10k max operations', itc2007_track2_files, 'itc2007_track2', ac3_params={}, depth_first_params={'max_operations': 10000})

experiment results will be saved in output/AC3 and depth_first_search with 10k max operations_itc2007_track2_20240321-163140
finished creating Q with 159600 elements


KeyboardInterrupt: 

In [ ]:
run_experiment('AC3 and depth_first_search with 10k max operations', itc2007_track3_files, 'itc2007_track3', ac3_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC3 and depth_first_search with 10k max operations', itc2019_files, 'itc2019', ac3_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2007_track2_files, 'itc2007_track2', ac4_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2007_track3_files, 'itc2007_track3', ac4_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2019_files, 'itc2019', ac4_params={}, depth_first_params={'max_operations': 10000})